In [38]:
from requests import get
import pandas as pd

In [39]:
europe_all = get("https://restcountries.com/v3.1/region/europe").json()

In [40]:
europe_countries_set = set()
for i in europe_all:
    europe_countries_set.add((i['name']['common'], i['subregion'], i['population']))

In [41]:
europe_countries_df = pd.DataFrame(europe_countries_set, columns=['name', 'subregion', 'population'])\
                        .sort_values('name')

In [42]:
# Where population is over 100,000
over_100k_pop = europe_countries_df[europe_countries_df['population'] > 100000].reset_index(drop=True)

In [43]:
over_100k_pop.sort_values('name').head()

,name,subregion,population
0,Albania,Southeast Europe,2837743
1,Austria,Central Europe,8917205
2,Belarus,Eastern Europe,9398861
3,Belgium,Western Europe,11555997
4,Bosnia and Herzegovina,Southeast Europe,3280815


In [44]:
# Top 5 biggest countries in terms of population
over_100k_pop.tail()

,name,subregion,population
37,Spain,Southern Europe,47351567
38,Sweden,Northern Europe,10353442
39,Switzerland,Western Europe,8654622
40,Ukraine,Eastern Europe,44134693
41,United Kingdom,Northern Europe,67215293


In [46]:
# Now import the EU list of countries (as well as other EU association from the european comission excel)
# Downloaded from the european comission website https://commission.europa.eu/index_en
european_comission = pd.read_excel("Country_Codes_and_Names.xlsx", sheet_name="Sheet1")

In [47]:
# Remove any brackets from the country name column
# Removes any space followed by a bracket followed by any character 0 or many times followed by another bracket
european_comission['COUNTRY NAME'] = european_comission['COUNTRY NAME'].replace(to_replace = ' (.*)', value = '', regex = True)

In [48]:
# We only care about countries in europe
european_comission = european_comission.iloc[1:36].reset_index(drop=True)
display(european_comission.head())
european_comission.tail()

,AREA,CODE,COUNTRY NAME
0,European Union (EU),BE,Belgium
1,European Union (EU),BG,Bulgaria
2,European Union (EU),CZ,Czech
3,European Union (EU),DK,Denmark
4,European Union (EU),DE,Germany


,AREA,CODE,COUNTRY NAME
30,European Free Trade Association (EFTA),NO,Norway
31,European Free Trade Association (EFTA),CH,Switzerland
32,EU candidate countries,TR,Turkey
33,European Neighbourhood Policy (ENP)-South,IL,Israel
34,Other European countries,RU,Russia


In [56]:
# Now merge both dataframes on a LEFT join where the original countries over 100k pop is on the left
european_countries_euflag_over100k = pd.merge(over_100k_pop, european_comission, left_on="name", right_on="COUNTRY NAME", how="left", validate='one_to_one')\
                                        [['name', 'subregion', 'population', 'AREA']]\
                                        .rename(columns={
                                            'name':'Country', 
                                            'subregion': 'Subregion', 
                                            'population': 'Poplation', 
                                            'AREA': 'European Union Association'
                                            })

In [60]:
european_countries_euflag_over100k['European Union Association'] = european_countries_euflag_over100k['European Union Association']\
                            .replace(to_replace='Other European countries', value='No Association').fillna("No Association")
european_countries_euflag_over100k

,Country,Subregion,Poplation,European Union Association
0,Albania,Southeast Europe,2837743,No Association
1,Austria,Central Europe,8917205,European Union (EU)
2,Belarus,Eastern Europe,9398861,No Association
3,Belgium,Western Europe,11555997,European Union (EU)
4,Bosnia and Herzegovina,Southeast Europe,3280815,No Association
5,Bulgaria,Southeast Europe,6927288,European Union (EU)
6,Croatia,Southeast Europe,4047200,European Union (EU)
7,Cyprus,Southern Europe,1207361,European Union (EU)
8,Czechia,Central Europe,10698896,No Association
9,Denmark,Northern Europe,5831404,European Union (EU)


In [62]:
european_countries_euflag_over100k.to_excel("countries_over_100k_pop.xlsx", index=False)